<a href="https://colab.research.google.com/github/Saranyanv/test/blob/master/test_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [0]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
  
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
X_train=X_train[0:600]
y_train=y_train[0:600]
X_test=X_test[0:100]
y_test=y_test[0:100]

In [0]:
import numpy as np
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[0:50], X_train[50:500]
y_valid, y_train = y_train[0:50], y_train[50:500]

In [0]:
n_inputs = 28*28  # MNIST
n_hidden1_1 = 300
n_hidden1_2 = 100
n_hidden2_1 = 300
n_hidden2_2 = 100
n_outputs = 10

In [0]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn1"):
    hidden1 = tf.layers.dense(X, n_hidden1_1, name="hidden1",
                              activation=tf.nn.softmax)
    hidden2 = tf.layers.dense(hidden1, n_hidden1_2, name="hidden2",
                              activation=tf.nn.softmax)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss1"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.01

with tf.name_scope("train1"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval1"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [51]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

0 Batch accuracy: 0.2 Validation accuracy: 0.14
1 Batch accuracy: 0.14 Validation accuracy: 0.14
2 Batch accuracy: 0.06 Validation accuracy: 0.14
3 Batch accuracy: 0.12 Validation accuracy: 0.14
4 Batch accuracy: 0.12 Validation accuracy: 0.14
5 Batch accuracy: 0.02 Validation accuracy: 0.14
6 Batch accuracy: 0.22 Validation accuracy: 0.14
7 Batch accuracy: 0.16 Validation accuracy: 0.14
8 Batch accuracy: 0.22 Validation accuracy: 0.14
9 Batch accuracy: 0.12 Validation accuracy: 0.14
10 Batch accuracy: 0.14 Validation accuracy: 0.14
11 Batch accuracy: 0.18 Validation accuracy: 0.14
12 Batch accuracy: 0.1 Validation accuracy: 0.14
13 Batch accuracy: 0.02 Validation accuracy: 0.14
14 Batch accuracy: 0.18 Validation accuracy: 0.14
15 Batch accuracy: 0.06 Validation accuracy: 0.14
16 Batch accuracy: 0.16 Validation accuracy: 0.14
17 Batch accuracy: 0.12 Validation accuracy: 0.14
18 Batch accuracy: 0.14 Validation accuracy: 0.14
19 Batch accuracy: 0.12 Validation accuracy: 0.14


In [0]:
#show_graph(tf.get_default_graph())
#show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn2"):
    hidden1 = tf.layers.dense(X, n_hidden1_1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden1_2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss2"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")   

In [0]:
learning_rate = 0.01

with tf.name_scope("train2"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval2"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [59]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

0 Batch accuracy: 0.16 Validation accuracy: 0.18
1 Batch accuracy: 0.12 Validation accuracy: 0.24
2 Batch accuracy: 0.2 Validation accuracy: 0.32
3 Batch accuracy: 0.38 Validation accuracy: 0.38
4 Batch accuracy: 0.6 Validation accuracy: 0.48
5 Batch accuracy: 0.56 Validation accuracy: 0.5
6 Batch accuracy: 0.64 Validation accuracy: 0.56
7 Batch accuracy: 0.66 Validation accuracy: 0.56
8 Batch accuracy: 0.66 Validation accuracy: 0.58
9 Batch accuracy: 0.66 Validation accuracy: 0.58
10 Batch accuracy: 0.78 Validation accuracy: 0.58
11 Batch accuracy: 0.82 Validation accuracy: 0.58
12 Batch accuracy: 0.74 Validation accuracy: 0.6
13 Batch accuracy: 0.68 Validation accuracy: 0.62
14 Batch accuracy: 0.76 Validation accuracy: 0.62
15 Batch accuracy: 0.84 Validation accuracy: 0.62
16 Batch accuracy: 0.84 Validation accuracy: 0.64
17 Batch accuracy: 0.74 Validation accuracy: 0.66
18 Batch accuracy: 0.86 Validation accuracy: 0.66
19 Batch accuracy: 0.8 Validation accuracy: 0.66


In [0]:
#show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn3"):
    hidden1 = tf.layers.dense(X, n_hidden2_1, name="hidden1",
                              activation=tf.nn.softmax)
    hidden2 = tf.layers.dense(hidden1, n_hidden2_2, name="hidden2",
                              activation=tf.nn.softmax)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss3"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [0]:
learning_rate = 0.01

with tf.name_scope("train3"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval3"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [67]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

0 Batch accuracy: 0.2 Validation accuracy: 0.14
1 Batch accuracy: 0.14 Validation accuracy: 0.14
2 Batch accuracy: 0.06 Validation accuracy: 0.14
3 Batch accuracy: 0.12 Validation accuracy: 0.14
4 Batch accuracy: 0.12 Validation accuracy: 0.14
5 Batch accuracy: 0.02 Validation accuracy: 0.14
6 Batch accuracy: 0.22 Validation accuracy: 0.14
7 Batch accuracy: 0.16 Validation accuracy: 0.14
8 Batch accuracy: 0.22 Validation accuracy: 0.14
9 Batch accuracy: 0.12 Validation accuracy: 0.14
10 Batch accuracy: 0.14 Validation accuracy: 0.14
11 Batch accuracy: 0.18 Validation accuracy: 0.14
12 Batch accuracy: 0.1 Validation accuracy: 0.14
13 Batch accuracy: 0.02 Validation accuracy: 0.14
14 Batch accuracy: 0.18 Validation accuracy: 0.14
15 Batch accuracy: 0.06 Validation accuracy: 0.14
16 Batch accuracy: 0.16 Validation accuracy: 0.14
17 Batch accuracy: 0.12 Validation accuracy: 0.14
18 Batch accuracy: 0.14 Validation accuracy: 0.14
19 Batch accuracy: 0.12 Validation accuracy: 0.14


In [0]:
#show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn4"):
    hidden1 = tf.layers.dense(X, n_hidden2_1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2_2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss4"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [0]:
learning_rate = 0.01

with tf.name_scope("train4"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval4"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [84]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

0 Batch accuracy: 0.16 Validation accuracy: 0.18
1 Batch accuracy: 0.12 Validation accuracy: 0.24
2 Batch accuracy: 0.2 Validation accuracy: 0.32
3 Batch accuracy: 0.38 Validation accuracy: 0.38
4 Batch accuracy: 0.6 Validation accuracy: 0.48
5 Batch accuracy: 0.56 Validation accuracy: 0.5
6 Batch accuracy: 0.64 Validation accuracy: 0.56
7 Batch accuracy: 0.66 Validation accuracy: 0.56
8 Batch accuracy: 0.66 Validation accuracy: 0.58
9 Batch accuracy: 0.66 Validation accuracy: 0.58
10 Batch accuracy: 0.78 Validation accuracy: 0.58
11 Batch accuracy: 0.82 Validation accuracy: 0.58
12 Batch accuracy: 0.74 Validation accuracy: 0.6
13 Batch accuracy: 0.68 Validation accuracy: 0.62
14 Batch accuracy: 0.76 Validation accuracy: 0.62
15 Batch accuracy: 0.84 Validation accuracy: 0.62
16 Batch accuracy: 0.84 Validation accuracy: 0.64
17 Batch accuracy: 0.74 Validation accuracy: 0.66
18 Batch accuracy: 0.86 Validation accuracy: 0.66
19 Batch accuracy: 0.8 Validation accuracy: 0.66


In [0]:
#show_graph(tf.get_default_graph().as_graph_def())